In [4]:
from datetime import datetime
import pandas as pd
import json
from tools import save_to_json
pd.set_option('display.max_columns', None)

In [5]:
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S')
df = pd.read_csv('../data/actual_data/raw/19.10.2022.csv', sep=';', parse_dates=['Arrival time', 'Arrival forecast', 'Departure time', 'Departure forecast'])
df = df.dropna(axis=0, subset=["Arrival time", "Arrival forecast"])
df['Stop name'] = df['Stop name'].str.slice(0,30)

In [6]:
df_sorted = df.sort_values(by=["Journey identifier", "Arrival time"])
df_sorted.head(10)

,Day of operation,Journey identifier,Operator ID,Operator abbreviation,Operator name,Product ID,Linie,Line Text,Rotation ID,Transport ID,Additional run TF,Cancelled TF,OPUIC,Stop name,Arrival time,Arrival forecast,Arrival forecast status,Departure time,Departure forecast,Departure forecast status,Non-stopping pass TF,Arrival delay,Departure delay,Geopos,lod
7051,19.10.22,85:11:1007:001,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,1007,IC4,NaN,IC,False,False,8503000,Zürich HB,2022-10-19 06:23:00,2022-10-19 06:23:36,REAL,NaT,NaT,NaN,False,False,False,"47.378176674223226, 8.540212349099065",http://lod.opentransportdata.swiss/didok/8503000
48957,19.10.22,85:11:1009:001,85:11,SBB,Schweizerische Bundesbahnen SBB,NaN,1009,IC4,NaN,IC,False,False,8503000,Zürich HB,2022-10-19 07:23:00,2022-10-19 07:22:48,REAL,NaT,NaT,NaN,False,False,False,"47.378176674223226, 8.540212349099065",http://lod.opentransportdata.swiss/didok/8503000
7049,19.10.22,85:11:100:001,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,100,ICE,NaN,ICE,False,False,8500090,Basel Bad Bf,2022-10-19 19:19:00,2022-10-19 19:33:00,PROGNOSE,2022-10-19 19:22:00,2022-10-19 19:35:00,PROGNOSE,False,True,True,"47.56730790569818, 7.606920418473086",http://lod.opentransportdata.swiss/didok/8500090
7054,19.10.22,85:11:1011:001,85:11,SBB,Schweizerische Bundesbahnen SBB,NaN,1011,IC4,NaN,IC,False,False,8503000,Zürich HB,2022-10-19 08:23:00,2022-10-19 08:24:00,REAL,NaT,NaT,NaN,False,False,False,"47.378176674223226, 8.540212349099065",http://lod.opentransportdata.swiss/didok/8503000
42485,19.10.22,85:11:101:002,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,101,ICE,NaN,ICE,False,False,8500090,Basel Bad Bf,2022-10-19 10:38:00,2022-10-19 10:37:00,PROGNOSE,2022-10-19 10:40:00,2022-10-19 10:40:00,PROGNOSE,False,False,False,"47.56730790569818, 7.606920418473086",http://lod.opentransportdata.swiss/didok/8500090
27060,19.10.22,85:11:101:002,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,101,ICE,NaN,ICE,False,False,8500010,Basel SBB,2022-10-19 10:47:00,2022-10-19 10:47:46,REAL,NaT,NaT,NaN,False,False,False,"47.5474120550501, 7.589562790156525",http://lod.opentransportdata.swiss/didok/8500010
27062,19.10.22,85:11:102:001,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,102,ICE,NaN,ICE,False,False,8500090,Basel Bad Bf,2022-10-19 17:19:00,2022-10-19 17:20:00,PROGNOSE,2022-10-19 17:22:00,2022-10-19 17:24:00,PROGNOSE,False,False,False,"47.56730790569818, 7.606920418473086",http://lod.opentransportdata.swiss/didok/8500090
1,19.10.22,85:11:103:002,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,103,ICE,NaN,ICE,False,False,8500010,Basel SBB,2022-10-19 12:47:00,2022-10-19 12:48:22,REAL,NaT,NaT,NaN,False,False,False,"47.5474120550501, 7.589562790156525",http://lod.opentransportdata.swiss/didok/8500010
42487,19.10.22,85:11:104:001,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,104,ICE,NaN,ICE,False,False,8500090,Basel Bad Bf,2022-10-19 15:19:00,2022-10-19 15:20:00,PROGNOSE,2022-10-19 15:23:00,2022-10-19 15:23:00,PROGNOSE,False,False,False,"47.56730790569818, 7.606920418473086",http://lod.opentransportdata.swiss/didok/8500090
3,19.10.22,85:11:10505:005,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,10505,IR15,NaN,IR,True,False,8507000,Bern,2022-10-19 05:56:00,2022-10-19 05:55:51,REAL,2022-10-19 06:00:00,2022-10-19 06:00:51,REAL,False,False,False,"46.948832290498416, 7.439130889923935",http://lod.opentransportdata.swiss/didok/8507000


## Train station position

In [7]:
df_all_stops = pd.read_csv('../data/raw/stops.csv', sep=',')
# Only take the stops that are also in the actual data. Remove the bus station and the duplicates.
df_stops = df_all_stops[df_all_stops['stop_name'].isin(df_sorted['Stop name'].unique())]
df_stops = df_stops.drop_duplicates(subset="stop_name")
stops = {}
for i, stop in df_stops.iterrows():
    stops[stop['stop_name']] = {'lat': stop['stop_lat'], 'lng': stop['stop_lon']}
save_to_json('../data/actual_data/clean/stops.json', stops)

## Retards par ligne
* Pour chaque row
  + Si le journey identifier est le même que le précédent
    - Prendre la gare des deux row, et y associer le retard de la 2ème row
        

In [8]:
df_merged = pd.concat([df_sorted, df_sorted.shift(-1).add_prefix('next_')], axis=1)

In [6]:
class DelayForLine:
    stationA: str
    stationB: str
    nbTrain: int
    nbDelayed: int
    totalDelay: int
    nbCancelled: int

    def __init__(self, stations):
        self.stationA = stations[0]
        self.stationB = stations[1]
        self.nbTrain = 0
        self.nbDelayed = 0
        self.totalDelay = 0
        self.nbCancelled = 0

    def add(self, is_delayed, delay, cancelled):
        self.nbTrain += 1
        self.nbDelayed += is_delayed
        self.totalDelay += delay # TODO maybe only if is_delayed is True
        self.nbCancelled += cancelled

    def __repr__(self):
        return "DelayForLine: " + json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)

In [7]:
delay_by_line = {}

for i, row in df_merged.iterrows():
    if row['Journey identifier'] == row['next_Journey identifier']:
        is_delayed = row['Arrival delay']
        delay = abs(row['Arrival forecast'] - row['Arrival time']).total_seconds()
        depart_station = row['Stop name']
        arrival_station = row['next_Stop name']
        cancelled = row['Cancelled TF']
        sorted_stations = sorted([depart_station, arrival_station])
        line = '|'.join(sorted_stations)
        if line not in delay_by_line:
            delay_by_line[line] = DelayForLine(sorted_stations)
        delay_by_line[line].add(is_delayed, delay, cancelled)


In [8]:
save_to_json('../data/actual_data/clean/delay_by_line.json', delay_by_line)

## Retards par gare
Pour chaque row
    Associer le retards à la gare

In [12]:
class DelayForStation:
    stationName: str
    stationGeopos: str # TODO maybe uncessary since we have the station infos already ?
    nbDelayed: int
    nbDelayed: int
    nbTrain: int
    totalDelay: int
    nbCancelled: int

    def __init__(self, stationName, stationGeopos):
        self.stationName = stationName
        self.stationGeopos = stationGeopos
        self.nbTrain = 0
        self.nbDelayed = 0
        self.totalDelay = 0
        self.nbCancelled = 0


    def add(self, is_delayed, delay, cancelled):
        self.nbTrain += 1
        self.nbDelayed += is_delayed
        self.totalDelay += delay # TODO maybe only if is_delayed is True
        self.nbCancelled += cancelled

    def __repr__(self):
        return "DelayForLine: " + json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)

In [13]:
delay_by_station = {}

for i, row in df_merged.iterrows():
    if row['Journey identifier'] == row['next_Journey identifier']:
        is_delayed = row['Arrival delay']
        delay = abs(row['Arrival forecast'] - row['Arrival time']).total_seconds()
        stationName = row['Stop name']
        stationGeopos = row['Geopos']
        cancelled = row['Cancelled TF']
        line = stationName
        if line not in delay_by_station:
            delay_by_station[line] = DelayForStation(stationName, stationGeopos)
        delay_by_station[line].add(is_delayed, delay, cancelled)

In [14]:
save_to_json('../data/actual_data/clean/delay_by_station.json', delay_by_station)

## Retard par jour de la semaine

## Retard par heure